- NNでやってみる

In [1]:
import warnings
import time
import sys
import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

warnings.simplefilter(action='ignore', category=FutureWarning)

import winsound
frequency = 400  # Hz
duration = 700    # milliseconds

pd.set_option('display.max_columns', None)

In [2]:
# def preprocessing(df):
#     for col in ['authorized_flag', 'category_1']:
#         df[col] = df[col].map({'Y':1, 'N':0})
#     for col in ['category_3']:
#         df[col] = df[col].map({'A':1, 'B':2,'C':3})
#     return df

# def read_data(input_file):
#     df = pd.read_csv(input_file)
#     df['first_active_month'] = pd.to_datetime(df['first_active_month'])
#     df['elapsed_time'] = (pd.Timestamp('2018-02-01') - df['first_active_month']).dt.days
#     return df

In [3]:
new_merchant_transactions = pd.read_csv('../Kaggle-data/new_merchant_transactions.csv', parse_dates=['purchase_date'])
historical_transactions = pd.read_csv('../Kaggle-data/historical_transactions.csv', parse_dates=['purchase_date'])
merchants = pd.read_csv('../Kaggle-data/merchants.csv')
train = pd.read_csv('../Kaggle-data/train.csv', parse_dates=['first_active_month'])
test = pd.read_csv('../Kaggle-data/test.csv', parse_dates=['first_active_month'])
winsound.Beep(frequency, duration)

In [39]:
# 問題が起きればここから回す
new = new_merchant_transactions.copy()
hist = historical_transactions.copy()
merchants_temp = merchants.copy()
# train_temp = train.copy()
# test_temp = test.copy()

In [40]:
train

,first_active_month,card_id,feature_1,feature_2,feature_3,target,elapsed_time,quarter,days_feature1,days_feature2,days_feature3,days_feature1_ratio,days_feature2_ratio,days_feature3_ratio,feature_sum,feature_product,feature_mean,feature_max,feature_min,feature_var
0,2017-06-01,C_ID_92a2005557,5,2,1,-0.820283,245,2,1225,490,245,0.020408,0.008163,0.004082,8,10,2.666667,5,1,2.081666
1,2017-01-01,C_ID_3d0044924f,4,1,0,0.392913,396,1,1584,396,0,0.010101,0.002525,0.000000,5,0,1.666667,4,0,2.081666
2,2016-08-01,C_ID_d639edf6cd,2,2,0,0.688056,549,3,1098,1098,0,0.003643,0.003643,0.000000,4,0,1.333333,2,0,1.154701
3,2017-09-01,C_ID_186d6a6901,4,3,0,0.142495,153,3,612,459,0,0.026144,0.019608,0.000000,7,0,2.333333,4,0,2.081666
4,2017-11-01,C_ID_cdbd2c0db2,1,3,0,-0.159749,92,4,92,276,0,0.010870,0.032609,0.000000,4,0,1.333333,3,0,1.527525
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201912,2017-09-01,C_ID_963962de2c,3,2,1,-2.740821,153,3,459,306,153,0.019608,0.013072,0.006536,6,6,2.000000,3,1,1.000000
201913,2015-10-01,C_ID_1314773c0b,3,1,1,0.312917,854,4,2562,854,854,0.003513,0.001171,0.001171,5,3,1.666667,3,1,1.154701
201914,2017-08-01,C_ID_7666735b3d,4,3,0,0.093494,184,3,736,552,0,0.021739,0.016304,0.000000,7,0,2.333333,4,0,2.081666
201915,2016-07-01,C_ID_73f5a0efd0,3,2,1,-4.676589,580,3,1740,1160,580,0.005172,0.003448,0.001724,6,6,2.000000,3,1,1.000000


In [41]:
for df in [test, train]:
    # https://www.kaggle.com/code/mfjwr1/simple-lightgbm-without-blending
    df['elapsed_time'] = (datetime.datetime.strptime('2018-02-01', '%Y-%m-%d') - df['first_active_month']).dt.days # 2018-02-01がmax
    df['quarter'] = df['first_active_month'].dt.quarter
    df['days_feature1'] = df['elapsed_time'] * df['feature_1']
    df['days_feature2'] = df['elapsed_time'] * df['feature_2']
    df['days_feature3'] = df['elapsed_time'] * df['feature_3']
    df['days_feature1_ratio'] = df['feature_1'] / df['elapsed_time']
    df['days_feature2_ratio'] = df['feature_2'] / df['elapsed_time']
    df['days_feature3_ratio'] = df['feature_3'] / df['elapsed_time']
    df['feature_sum'] = df['feature_1'] + df['feature_2'] + df['feature_3']
    df['feature_product'] = df['feature_1'] * df['feature_2'] * df['feature_3'] # Kawai added
    df['feature_mean'] = df['feature_sum'] / 3
    df['feature_max'] = df[['feature_1', 'feature_2', 'feature_3']].max(axis=1)
    df['feature_min'] = df[['feature_1', 'feature_2', 'feature_3']].min(axis=1)
    df['feature_var'] = df[['feature_1', 'feature_2', 'feature_3']].std(axis=1)

In [42]:
train

,first_active_month,card_id,feature_1,feature_2,feature_3,target,elapsed_time,quarter,days_feature1,days_feature2,days_feature3,days_feature1_ratio,days_feature2_ratio,days_feature3_ratio,feature_sum,feature_product,feature_mean,feature_max,feature_min,feature_var
0,2017-06-01,C_ID_92a2005557,5,2,1,-0.820283,245,2,1225,490,245,0.020408,0.008163,0.004082,8,10,2.666667,5,1,2.081666
1,2017-01-01,C_ID_3d0044924f,4,1,0,0.392913,396,1,1584,396,0,0.010101,0.002525,0.000000,5,0,1.666667,4,0,2.081666
2,2016-08-01,C_ID_d639edf6cd,2,2,0,0.688056,549,3,1098,1098,0,0.003643,0.003643,0.000000,4,0,1.333333,2,0,1.154701
3,2017-09-01,C_ID_186d6a6901,4,3,0,0.142495,153,3,612,459,0,0.026144,0.019608,0.000000,7,0,2.333333,4,0,2.081666
4,2017-11-01,C_ID_cdbd2c0db2,1,3,0,-0.159749,92,4,92,276,0,0.010870,0.032609,0.000000,4,0,1.333333,3,0,1.527525
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201912,2017-09-01,C_ID_963962de2c,3,2,1,-2.740821,153,3,459,306,153,0.019608,0.013072,0.006536,6,6,2.000000,3,1,1.000000
201913,2015-10-01,C_ID_1314773c0b,3,1,1,0.312917,854,4,2562,854,854,0.003513,0.001171,0.001171,5,3,1.666667,3,1,1.154701
201914,2017-08-01,C_ID_7666735b3d,4,3,0,0.093494,184,3,736,552,0,0.021739,0.016304,0.000000,7,0,2.333333,4,0,2.081666
201915,2016-07-01,C_ID_73f5a0efd0,3,2,1,-4.676589,580,3,1740,1160,580,0.005172,0.003448,0.001724,6,6,2.000000,3,1,1.000000


In [43]:
test

,first_active_month,card_id,feature_1,feature_2,feature_3,elapsed_time,quarter,days_feature1,days_feature2,days_feature3,days_feature1_ratio,days_feature2_ratio,days_feature3_ratio,feature_sum,feature_product,feature_mean,feature_max,feature_min,feature_var
0,2017-04-01,C_ID_0ab67a22ab,3,3,1,306.0,2.0,918.0,918.0,306.0,0.009804,0.009804,0.003268,7,9,2.333333,3,1,1.154701
1,2017-01-01,C_ID_130fd0cbdd,2,3,0,396.0,1.0,792.0,1188.0,0.0,0.005051,0.007576,0.000000,5,0,1.666667,3,0,1.527525
2,2017-08-01,C_ID_b709037bc5,5,1,1,184.0,3.0,920.0,184.0,184.0,0.027174,0.005435,0.005435,7,5,2.333333,5,1,2.309401
3,2017-12-01,C_ID_d27d835a9f,2,1,0,62.0,4.0,124.0,62.0,0.0,0.032258,0.016129,0.000000,3,0,1.000000,2,0,1.000000
4,2015-12-01,C_ID_2b5e3df5c2,5,1,1,793.0,4.0,3965.0,793.0,793.0,0.006305,0.001261,0.001261,7,5,2.333333,5,1,2.309401
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123618,2017-10-01,C_ID_7a239d2eda,2,2,0,123.0,4.0,246.0,246.0,0.0,0.016260,0.016260,0.000000,4,0,1.333333,2,0,1.154701
123619,2017-09-01,C_ID_75ace375ae,3,1,1,153.0,3.0,459.0,153.0,153.0,0.019608,0.006536,0.006536,5,3,1.666667,3,1,1.154701
123620,2016-09-01,C_ID_21d56d950c,5,1,1,518.0,3.0,2590.0,518.0,518.0,0.009653,0.001931,0.001931,7,5,2.333333,5,1,2.309401
123621,2017-06-01,C_ID_6c46fc5a9d,2,1,0,245.0,2.0,490.0,245.0,0.0,0.008163,0.004082,0.000000,3,0,1.000000,2,0,1.000000


In [44]:
# installmentsが999のものを削除
new = new[new['installments'] != 999]
hist = hist[hist['installments'] != 999]

In [45]:
def encode_categorical_columns(df, columns):
    return pd.get_dummies(df, columns=columns)

new = encode_categorical_columns(new, ['category_2', 'category_3']) # 0625 state_id追加したが悪くなった
hist = encode_categorical_columns(hist, ['category_2', 'category_3'])
merchants_temp = encode_categorical_columns(merchants_temp, ['category_2', 'most_recent_sales_range', 'most_recent_purchases_range'])

In [46]:
merchants_temp['category_1'] = merchants_temp['category_1'].map(lambda x: 1 if x=='Y' else 0)
merchants_temp['category_4'] = merchants_temp['category_4'].map(lambda x: 1 if x=='Y' else 0)
# merchants_temp

In [47]:
hist

,authorized_flag,card_id,city_id,category_1,installments,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,state_id,subsector_id,category_2_1.0,category_2_2.0,category_2_3.0,category_2_4.0,category_2_5.0,category_3_A,category_3_B,category_3_C
0,Y,C_ID_4e6213e9bc,88,N,0,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,16,37,True,False,False,False,False,True,False,False
1,Y,C_ID_4e6213e9bc,88,N,0,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,16,16,True,False,False,False,False,True,False,False
2,Y,C_ID_4e6213e9bc,88,N,0,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09 22:04:29,16,37,True,False,False,False,False,True,False,False
3,Y,C_ID_4e6213e9bc,88,N,0,560,M_ID_e6d5ae8ea6,-5,-0.735352,2017-09-02 10:06:26,16,34,True,False,False,False,False,True,False,False
4,Y,C_ID_4e6213e9bc,88,N,0,80,M_ID_e020e9b302,-11,-0.722865,2017-03-10 01:14:19,16,37,True,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29112356,Y,C_ID_2863d2fa95,-1,Y,1,360,M_ID_edd92b6720,-1,-0.632706,2017-01-20 08:52:04,-1,34,False,False,False,False,False,False,True,False
29112357,Y,C_ID_2863d2fa95,-1,Y,1,360,M_ID_edd92b6720,0,-0.632706,2017-02-20 04:40:50,-1,34,False,False,False,False,False,False,True,False
29112358,Y,C_ID_5c240d6e3c,3,N,0,278,M_ID_9cdcfe8673,0,-0.657740,2017-12-26 18:37:51,16,37,True,False,False,False,False,True,False,False
29112359,Y,C_ID_5c240d6e3c,331,N,0,514,M_ID_1a75f94f92,-1,0.770620,2017-11-24 14:18:15,16,9,True,False,False,False,False,True,False,False


In [48]:
new

,authorized_flag,card_id,city_id,category_1,installments,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,state_id,subsector_id,category_2_1.0,category_2_2.0,category_2_3.0,category_2_4.0,category_2_5.0,category_3_A,category_3_B,category_3_C
0,Y,C_ID_415bb3a509,107,N,1,307,M_ID_b0c793002c,1,-0.557574,2018-03-11 14:57:36,9,19,True,False,False,False,False,False,True,False
1,Y,C_ID_415bb3a509,140,N,1,307,M_ID_88920c89e8,1,-0.569580,2018-03-19 18:53:37,9,19,True,False,False,False,False,False,True,False
2,Y,C_ID_415bb3a509,330,N,1,507,M_ID_ad5237ef6b,2,-0.551037,2018-04-26 14:08:44,9,14,True,False,False,False,False,False,True,False
3,Y,C_ID_415bb3a509,-1,Y,1,661,M_ID_9e84cda3b1,1,-0.671925,2018-03-07 09:43:21,-1,8,False,False,False,False,False,False,True,False
4,Y,C_ID_ef55cf8d4b,-1,Y,1,166,M_ID_3c86fa3831,1,-0.659904,2018-03-22 21:07:53,-1,29,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1963026,Y,C_ID_1320dee851,142,N,0,309,M_ID_7754b67f3b,2,-0.701828,2018-04-06 14:36:52,19,21,False,False,True,False,False,True,False,False
1963027,Y,C_ID_f112aa3381,158,N,0,560,M_ID_da063195b7,2,-0.694390,2018-03-07 13:19:18,15,34,True,False,False,False,False,True,False,False
1963028,Y,C_ID_bd97b86450,69,N,1,278,M_ID_9a9ccb6544,1,-0.621031,2018-03-05 12:04:56,9,37,True,False,False,False,False,False,True,False
1963029,Y,C_ID_c0513fd84f,130,N,0,367,M_ID_40c28d596f,1,-0.656749,2018-03-09 14:47:05,7,16,False,False,True,False,False,True,False,False


In [49]:
bool_columns = ['category_2_1.0', 'category_2_2.0', 'category_2_3.0', 'category_2_4.0', 'category_2_5.0', 'category_3_A', 'category_3_B', 'category_3_C'
                # 'state_id_-1', 'state_id_1', 'state_id_2', 'state_id_3', 'state_id_4', 'state_id_5', 'state_id_6', 'state_id_7', 'state_id_8', 'state_id_9',
                # 'state_id_10', 'state_id_11', 'state_id_12', 'state_id_13', 'state_id_14', 'state_id_15', 'state_id_16', 'state_id_17', 'state_id_18', 'state_id_19',
                # 'state_id_20', 'state_id_21', 'state_id_22', 'state_id_23', 'state_id_24'
                ]
new[bool_columns] = new[bool_columns].map(lambda x: 1 if x == True else 0)
hist[bool_columns] = hist[bool_columns].map(lambda x: 1 if x == True else 0)

In [50]:
# bool_columns_for_merchants = ['category_2_1.0', 'category_2_2.0', 'category_2_3.0', 'category_2_4.0', 'category_2_5.0',
#                               'most_recent_sales_range_A', 'most_recent_sales_range_B', 'most_recent_sales_range_C', 'most_recent_sales_range_D', 'most_recent_sales_range_E',
#                               'most_recent_purchases_range_A', 'most_recent_purchases_range_B', 'most_recent_purchases_range_C', 'most_recent_purchases_range_D', 'most_recent_purchases_range_E']

# merchants_temp[bool_columns_for_merchants] = merchants_temp[bool_columns_for_merchants].map(lambda x: 1 if x == True else 0)

In [51]:
new_hist = pd.concat([new, hist], axis=0)

In [52]:
new_hist

,authorized_flag,card_id,city_id,category_1,installments,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,state_id,subsector_id,category_2_1.0,category_2_2.0,category_2_3.0,category_2_4.0,category_2_5.0,category_3_A,category_3_B,category_3_C
0,Y,C_ID_415bb3a509,107,N,1,307,M_ID_b0c793002c,1,-0.557574,2018-03-11 14:57:36,9,19,1,0,0,0,0,0,1,0
1,Y,C_ID_415bb3a509,140,N,1,307,M_ID_88920c89e8,1,-0.569580,2018-03-19 18:53:37,9,19,1,0,0,0,0,0,1,0
2,Y,C_ID_415bb3a509,330,N,1,507,M_ID_ad5237ef6b,2,-0.551037,2018-04-26 14:08:44,9,14,1,0,0,0,0,0,1,0
3,Y,C_ID_415bb3a509,-1,Y,1,661,M_ID_9e84cda3b1,1,-0.671925,2018-03-07 09:43:21,-1,8,0,0,0,0,0,0,1,0
4,Y,C_ID_ef55cf8d4b,-1,Y,1,166,M_ID_3c86fa3831,1,-0.659904,2018-03-22 21:07:53,-1,29,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29112356,Y,C_ID_2863d2fa95,-1,Y,1,360,M_ID_edd92b6720,-1,-0.632706,2017-01-20 08:52:04,-1,34,0,0,0,0,0,0,1,0
29112357,Y,C_ID_2863d2fa95,-1,Y,1,360,M_ID_edd92b6720,0,-0.632706,2017-02-20 04:40:50,-1,34,0,0,0,0,0,0,1,0
29112358,Y,C_ID_5c240d6e3c,3,N,0,278,M_ID_9cdcfe8673,0,-0.657740,2017-12-26 18:37:51,16,37,1,0,0,0,0,1,0,0
29112359,Y,C_ID_5c240d6e3c,331,N,0,514,M_ID_1a75f94f92,-1,0.770620,2017-11-24 14:18:15,16,9,1,0,0,0,0,1,0,0


In [53]:
new_hist['purchase_amount_new'] = np.round(new_hist['purchase_amount'] / 0.00150265118 + 497.06)

In [54]:
new_hist = new_hist.drop('purchase_amount', axis=1)

In [55]:
new_hist['category_1'] = new_hist['category_1'].map(lambda x: 1 if x =='Y' else 0)

In [56]:
new_hist['authorized_flag'] = new_hist['authorized_flag'].map(lambda x: 1 if x =='Y' else 0)

In [57]:
new_hist_features = new_hist.groupby('card_id').agg(authorized_ratio = ('authorized_flag', 'mean'),
                                                    cat_1_mean = ('category_1', 'mean'),
                                                    installments_mean = ('installments', 'mean'),
                                                    installments_max = ('installments', 'max'),
                                                    installments_min = ('installments', 'min'),
                                                    month_lag_mean = ('month_lag', 'mean'),
                                                    month_lag_max = ('month_lag', 'max'),
                                                    month_lag_min = ('month_lag', 'min'),
                                                    purchase_amount_mean = ('purchase_amount_new', 'mean'),
                                                    purchase_amount_max = ('purchase_amount_new', 'max'),
                                                    purchase_amount_min = ('purchase_amount_new', 'min'),
                                                    cat_2_1_mean = ('category_2_1.0', 'mean'),
                                                    cat_2_2_mean = ('category_2_2.0', 'mean'),
                                                    cat_2_3_mean = ('category_2_3.0', 'mean'),
                                                    cat_2_4_mean = ('category_2_4.0', 'mean'),
                                                    cat_2_5_mean = ('category_2_5.0', 'mean'),
                                                    cat_3_A_mean = ('category_3_A', 'mean'),
                                                    cat_3_B_mean = ('category_3_B', 'mean'),
                                                    cat_3_C_mean = ('category_3_C', 'mean'),
                                                    purchase_freq = ('purchase_amount_new', 'size')
                                                    # state_minus1count = ('state_id_-1', 'sum'),
                                                    # state_1count = ('state_id_1', 'sum'),
                                                    # state_2count = ('state_id_2', 'sum'),
                                                    # state_3count = ('state_id_3', 'sum'),
                                                    # state_4count = ('state_id_4', 'sum'),
                                                    # state_5count = ('state_id_5', 'sum'),
                                                    # state_6count = ('state_id_6', 'sum'),
                                                    # state_7count = ('state_id_7', 'sum'),
                                                    # state_8count = ('state_id_8', 'sum'),
                                                    # state_9count = ('state_id_9', 'sum'),
                                                    # state_10count = ('state_id_10', 'sum'),
                                                    # state_11count = ('state_id_11', 'sum'),
                                                    # state_12count = ('state_id_12', 'sum'),
                                                    # state_13count = ('state_id_13', 'sum'),
                                                    # state_14count = ('state_id_14', 'sum'),
                                                    # state_15count = ('state_id_15', 'sum'),
                                                    # state_16count = ('state_id_16', 'sum'),
                                                    # state_17count = ('state_id_17', 'sum'),
                                                    # state_18count = ('state_id_18', 'sum'),
                                                    # state_19count = ('state_id_19', 'sum'),
                                                    # state_20count = ('state_id_20', 'sum'),
                                                    # state_21count = ('state_id_21', 'sum'),
                                                    # state_22count = ('state_id_22', 'sum'),
                                                    # state_23count = ('state_id_23', 'sum'),
                                                    # state_24count = ('state_id_24', 'sum'),
                                                    ).reset_index()

In [58]:
new_hist_features

,card_id,authorized_ratio,cat_1_mean,installments_mean,installments_max,installments_min,month_lag_mean,month_lag_max,month_lag_min,purchase_amount_mean,purchase_amount_max,purchase_amount_min,cat_2_1_mean,cat_2_2_mean,cat_2_3_mean,cat_2_4_mean,cat_2_5_mean,cat_3_A_mean,cat_3_B_mean,cat_3_C_mean,purchase_freq
0,C_ID_00007093c1,0.768212,0.185430,1.284768,6,1,-5.748344,2,-12,152.516556,1500.0,12.0,0.006623,0.000000,0.801325,0.000000,0.006623,0.000000,0.841060,0.158940,151
1,C_ID_0001238066,0.979866,0.026846,1.610738,10,-1,-1.261745,2,-5,107.161074,1008.0,4.0,0.771812,0.000000,0.000000,0.000000,0.154362,0.000000,0.731544,0.241611,149
2,C_ID_0001506ef0,0.941176,0.000000,0.014706,1,0,-4.661765,1,-13,144.264706,1491.0,4.0,0.029412,0.000000,0.970588,0.000000,0.000000,0.985294,0.014706,0.000000,68
3,C_ID_0001793786,0.890688,0.008097,0.020243,1,0,-2.744939,2,-9,397.352227,3528.0,1.0,0.105263,0.340081,0.080972,0.000000,0.004049,0.979757,0.020243,0.000000,247
4,C_ID_000183fdda,0.954839,0.025806,1.806452,10,-1,-2.187097,2,-6,173.212903,2337.0,6.0,0.045161,0.006452,0.916129,0.000000,0.006452,0.000000,0.703226,0.264516,155
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325535,C_ID_ffff1d9928,0.812500,0.187500,1.687500,10,-1,-1.125000,2,-4,83.750000,500.0,0.0,0.062500,0.000000,0.000000,0.750000,0.000000,0.000000,0.812500,0.125000,16
325536,C_ID_ffff579d3a,0.869565,0.000000,0.026087,1,0,-3.695652,1,-6,28.573913,170.0,5.0,0.991304,0.000000,0.000000,0.000000,0.008696,0.973913,0.026087,0.000000,115
325537,C_ID_ffff756266,0.600000,0.120000,3.160000,12,-1,-8.160000,2,-13,360.640000,1300.0,0.0,0.880000,0.000000,0.000000,0.000000,0.000000,0.000000,0.440000,0.520000,25
325538,C_ID_ffff828181,0.878788,0.055556,1.202020,4,-1,-5.767677,2,-13,76.717172,570.0,1.0,0.095960,0.787879,0.000000,0.045455,0.005051,0.000000,0.873737,0.116162,198


In [59]:
# merchants_features = merchants_temp.groupby('merchant_id').agg(
#                                           numerical_1_mean = ('numerical_1', 'mean'),
#                                           numerical_1_max = ('numerical_1', 'max'),
#                                           numerical_1_min = ('numerical_1', 'min'),
#                                           numerical_2_mean = ('numerical_2', 'mean'),
#                                           numerical_2_max = ('numerical_2', 'max'),
#                                           numerical_2_min = ('numerical_2', 'min'),
#                                           cat_1_mean = ('category_1', 'mean'),
#                                           cat_4_mean = ('category_4', 'mean'),
#                                           cat_4_max = ('category_4', 'max'),
#                                           cat_4_min = ('category_4', 'min'),
#                                           avg_sales_lag3_mean = ('avg_sales_lag3', 'mean'),
#                                           avg_sales_lag3_max = ('avg_sales_lag3', 'max'),
#                                           avg_sales_lag3_min = ('avg_sales_lag3', 'min'),
#                                           avg_sales_lag6_mean = ('avg_sales_lag6', 'mean'),
#                                           avg_sales_lag6_max = ('avg_sales_lag6', 'max'),
#                                           avg_sales_lag6_min = ('avg_sales_lag6', 'min'),
#                                           avg_sales_lag12_mean = ('avg_sales_lag12', 'mean'),
#                                           avg_sales_lag12_max = ('avg_sales_lag12', 'max'),
#                                           avg_sales_lag12_min = ('avg_sales_lag12', 'min'),
                                          # most_recent_sales_A_mean = ('most_recent_sales_range_A', 'mean'),
                                          # most_recent_sales_A_max = ('most_recent_sales_range_A', 'max'),
                                          # most_recent_sales_A_min = ('most_recent_sales_range_A', 'min'),
                                          # most_recent_sales_B_mean = ('most_recent_sales_range_B', 'mean'),
                                          # most_recent_sales_B_max = ('most_recent_sales_range_B', 'max'),
                                          # most_recent_sales_B_min = ('most_recent_sales_range_B', 'min'),
                                          # most_recent_sales_C_mean = ('most_recent_sales_range_C', 'mean'),
                                          # most_recent_sales_C_max = ('most_recent_sales_range_C', 'max'),
                                          # most_recent_sales_C_min = ('most_recent_sales_range_C', 'min'),
                                          # most_recent_sales_D_mean = ('most_recent_sales_range_D', 'mean'),
                                          # most_recent_sales_D_max = ('most_recent_sales_range_D', 'max'),
                                          # most_recent_sales_D_min = ('most_recent_sales_range_D', 'min'),
                                          # most_recent_sales_E_mean = ('most_recent_sales_range_E', 'mean'),
                                          # most_recent_sales_E_max = ('most_recent_sales_range_E', 'max'),
                                          # most_recent_sales_E_min = ('most_recent_sales_range_E', 'min'),
                                          # most_recent_purchases_A_mean = ('most_recent_purchases_range_A', 'mean'),
                                          # most_recent_purchases_A_max = ('most_recent_purchases_range_A', 'max'),
                                          # most_recent_purchases_A_min = ('most_recent_purchases_range_A', 'min'),
                                          # most_recent_purchases_B_mean = ('most_recent_purchases_range_B', 'mean'),
                                          # most_recent_purchases_B_max = ('most_recent_purchases_range_B', 'max'),
                                          # most_recent_purchases_B_min = ('most_recent_purchases_range_B', 'min'),
                                          # most_recent_purchases_C_mean = ('most_recent_purchases_range_C', 'mean'),
                                          # most_recent_purchases_C_max = ('most_recent_purchases_range_C', 'max'),
                                          # most_recent_purchases_C_min = ('most_recent_purchases_range_C', 'min'),
                                          # most_recent_purchases_D_mean = ('most_recent_purchases_range_D', 'mean'),
                                          # most_recent_purchases_D_max = ('most_recent_purchases_range_D', 'max'),
                                          # most_recent_purchases_D_min = ('most_recent_purchases_range_D', 'min'),
                                          # most_recent_purchases_E_mean = ('most_recent_purchases_range_E', 'mean'),
                                          # most_recent_purchases_E_max = ('most_recent_purchases_range_E', 'max'),
                                          # most_recent_purchases_E_min = ('most_recent_purchases_range_E', 'min'),
                                          # cat_2_1_mean = ('category_2_1.0', 'mean'),
                                          # cat_2_2_mean = ('category_2_2.0', 'mean'),
                                          # cat_2_3_mean = ('category_2_3.0', 'mean'),
                                          # cat_2_4_mean = ('category_2_4.0', 'mean'),
                                          # cat_2_5_mean = ('category_2_5.0', 'mean')
                                          # ).reset_index()

In [60]:
df_train = pd.merge(train, new_hist_features, on='card_id')

In [61]:
df_test = pd.merge(test, new_hist_features, on='card_id')

In [62]:
df_train

,first_active_month,card_id,feature_1,feature_2,feature_3,target,elapsed_time,quarter,days_feature1,days_feature2,days_feature3,days_feature1_ratio,days_feature2_ratio,days_feature3_ratio,feature_sum,feature_product,feature_mean,feature_max,feature_min,feature_var,authorized_ratio,cat_1_mean,installments_mean,installments_max,installments_min,month_lag_mean,month_lag_max,month_lag_min,purchase_amount_mean,purchase_amount_max,purchase_amount_min,cat_2_1_mean,cat_2_2_mean,cat_2_3_mean,cat_2_4_mean,cat_2_5_mean,cat_3_A_mean,cat_3_B_mean,cat_3_C_mean,purchase_freq
0,2017-06-01,C_ID_92a2005557,5,2,1,-0.820283,245,2,1225,490,245,0.020408,0.008163,0.004082,8,10,2.666667,5,1,2.081666,0.954064,0.000000,0.014134,1,0,-3.473498,2,-8,75.628975,2000.0,5.0,0.989399,0.0,0.000000,0.000000,0.010601,0.985866,0.014134,0.000000,283
1,2017-01-01,C_ID_3d0044924f,4,1,0,0.392913,396,1,1584,396,0,0.010101,0.002525,0.000000,5,0,1.666667,4,0,2.081666,0.969101,0.087079,1.542135,10,-1,-4.921348,2,-12,96.345506,3578.0,3.0,0.912921,0.0,0.000000,0.000000,0.000000,0.000000,0.792135,0.202247,356
2,2016-08-01,C_ID_d639edf6cd,2,2,0,0.688056,549,3,1098,1098,0,0.003643,0.003643,0.000000,4,0,1.333333,2,0,1.154701,0.954545,0.000000,0.000000,0,0,-8.363636,2,-13,45.363636,400.0,11.0,0.090909,0.0,0.000000,0.000000,0.909091,1.000000,0.000000,0.000000,44
3,2017-09-01,C_ID_186d6a6901,4,3,0,0.142495,153,3,612,459,0,0.026144,0.019608,0.000000,7,0,2.333333,4,0,2.081666,1.000000,0.154762,1.059524,3,-1,-2.452381,2,-5,68.142857,1459.0,4.0,0.142857,0.0,0.000000,0.702381,0.000000,0.000000,0.880952,0.083333,84
4,2017-11-01,C_ID_cdbd2c0db2,1,3,0,-0.159749,92,4,92,276,0,0.010870,0.032609,0.000000,4,0,1.333333,3,0,1.527525,0.970414,0.100592,1.284024,12,-1,-0.680473,2,-3,226.887574,5284.0,0.0,0.071006,0.0,0.041420,0.781065,0.005917,0.000000,0.946746,0.047337,169
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201912,2017-09-01,C_ID_963962de2c,3,2,1,-2.740821,153,3,459,306,153,0.019608,0.013072,0.006536,6,6,2.000000,3,1,1.000000,0.914894,0.000000,0.021277,1,0,-2.000000,1,-5,25.297872,116.0,3.0,0.297872,0.0,0.702128,0.000000,0.000000,0.978723,0.021277,0.000000,47
201913,2015-10-01,C_ID_1314773c0b,3,1,1,0.312917,854,4,2562,854,854,0.003513,0.001171,0.001171,5,3,1.666667,3,1,1.154701,0.854167,0.000000,0.020833,1,0,-3.770833,1,-13,23.145833,115.0,2.0,1.000000,0.0,0.000000,0.000000,0.000000,0.979167,0.020833,0.000000,48
201914,2017-08-01,C_ID_7666735b3d,4,3,0,0.093494,184,3,736,552,0,0.021739,0.016304,0.000000,7,0,2.333333,4,0,2.081666,0.933333,0.100000,1.255556,9,-1,-3.577778,1,-6,246.388889,4500.0,6.0,0.055556,0.0,0.755556,0.000000,0.000000,0.000000,0.866667,0.122222,90
201915,2016-07-01,C_ID_73f5a0efd0,3,2,1,-4.676589,580,3,1740,1160,580,0.005172,0.003448,0.001724,6,6,2.000000,3,1,1.000000,0.935484,0.000000,0.000000,0,0,-2.548387,2,-9,154.419355,2000.0,10.0,1.000000,0.0,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,31


In [63]:
df_train[['days_feature1_ratio', 'days_feature2_ratio']] = df_train[['days_feature1_ratio', 'days_feature2_ratio']].replace(np.inf, np.nan)

In [64]:
df_train.isnull().sum()

first_active_month      0
card_id                 0
feature_1               0
feature_2               0
feature_3               0
target                  0
elapsed_time            0
quarter                 0
days_feature1           0
days_feature2           0
days_feature3           0
days_feature1_ratio     1
days_feature2_ratio     1
days_feature3_ratio     1
feature_sum             0
feature_product         0
feature_mean            0
feature_max             0
feature_min             0
feature_var             0
authorized_ratio        0
cat_1_mean              0
installments_mean       0
installments_max        0
installments_min        0
month_lag_mean          0
month_lag_max           0
month_lag_min           0
purchase_amount_mean    0
purchase_amount_max     0
purchase_amount_min     0
cat_2_1_mean            0
cat_2_2_mean            0
cat_2_3_mean            0
cat_2_4_mean            0
cat_2_5_mean            0
cat_3_A_mean            0
cat_3_B_mean            0
cat_3_C_mean

In [65]:
numeric_cols = df_train.select_dtypes(include=['float64', 'int64']).columns
numeric_cols = numeric_cols.drop('target')
# MinMaxScalerを使用して正規化
scaler = MinMaxScaler()
df_train[numeric_cols] = scaler.fit_transform(df_train[numeric_cols])

In [66]:
numeric_cols = df_test.select_dtypes(include=['float64', 'int64']).columns
# MinMaxScalerを使用して正規化
scaler = MinMaxScaler()
df_test[numeric_cols] = scaler.fit_transform(df_test[numeric_cols])

In [67]:
df_train

,first_active_month,card_id,feature_1,feature_2,feature_3,target,elapsed_time,quarter,days_feature1,days_feature2,days_feature3,days_feature1_ratio,days_feature2_ratio,days_feature3_ratio,feature_sum,feature_product,feature_mean,feature_max,feature_min,feature_var,authorized_ratio,cat_1_mean,installments_mean,installments_max,installments_min,month_lag_mean,month_lag_max,month_lag_min,purchase_amount_mean,purchase_amount_max,purchase_amount_min,cat_2_1_mean,cat_2_2_mean,cat_2_3_mean,cat_2_4_mean,cat_2_5_mean,cat_3_A_mean,cat_3_B_mean,cat_3_C_mean,purchase_freq
0,2017-06-01,C_ID_92a2005557,1.00,0.5,1.0,-0.820283,0.107268,2,0.157759,0.071512,0.107268,0.121699,0.080192,0.126531,1.000000,1.0,1.000000,1.00,1.0,0.868517,0.952619,0.000000,0.036203,0.083333,0.090909,0.639443,1.000000,0.416667,3.544876e-07,4.997500e-07,0.000256,0.989399,0.0,0.000000,0.000000,0.010601,0.985866,0.014134,0.000000,0.096464
1,2017-01-01,C_ID_3d0044924f,0.75,0.0,0.0,0.392913,0.173380,1,0.203992,0.057793,0.000000,0.057441,0.021668,0.000000,0.500000,0.0,0.500000,0.75,0.0,0.868517,0.968129,0.087079,0.161156,0.833333,0.000000,0.535782,1.000000,0.083333,4.528911e-07,8.942500e-07,0.000154,0.912921,0.0,0.000000,0.000000,0.000000,0.000000,0.792135,0.202247,0.121524
2,2016-08-01,C_ID_d639edf6cd,0.25,0.5,0.0,0.688056,0.240368,3,0.141404,0.160245,0.000000,0.017180,0.033270,0.000000,0.333333,0.0,0.333333,0.25,0.0,0.333333,0.953116,0.000000,0.035047,0.000000,0.090909,0.289328,1.000000,0.000000,2.107272e-07,9.975000e-08,0.000564,0.090909,0.0,0.000000,0.000000,0.909091,1.000000,0.000000,0.000000,0.014418
3,2017-09-01,C_ID_186d6a6901,0.75,1.0,0.0,0.142495,0.066988,3,0.078815,0.066988,0.000000,0.157456,0.198990,0.000000,0.833333,0.0,0.833333,0.75,0.0,0.868517,1.000000,0.154762,0.121690,0.250000,0.000000,0.712551,1.000000,0.666667,3.189285e-07,3.645000e-07,0.000205,0.142857,0.0,0.000000,0.702381,0.000000,0.000000,0.880952,0.083333,0.028150
4,2017-11-01,C_ID_cdbd2c0db2,0.00,1.0,0.0,-0.159749,0.040280,4,0.011848,0.040280,0.000000,0.062232,0.333943,0.000000,0.333333,0.0,0.333333,0.50,0.0,0.548584,0.969484,0.100592,0.140049,1.000000,0.000000,0.839412,1.000000,0.833333,1.072966e-06,1.320750e-06,0.000000,0.071006,0.0,0.041420,0.781065,0.005917,0.000000,0.946746,0.047337,0.057329
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201912,2017-09-01,C_ID_963962de2c,0.50,0.5,1.0,-2.740821,0.066988,3,0.059111,0.044658,0.066988,0.116709,0.131145,0.202614,0.666667,0.6,0.666667,0.50,1.0,0.244017,0.912217,0.000000,0.036787,0.083333,0.090909,0.744939,0.923077,0.666667,1.154149e-07,2.875000e-08,0.000154,0.297872,0.0,0.702128,0.000000,0.000000,0.978723,0.021277,0.000000,0.015448
201913,2015-10-01,C_ID_1314773c0b,0.50,0.0,1.0,0.312917,0.373905,4,0.329942,0.124635,0.373905,0.016369,0.007610,0.036300,0.500000,0.3,0.500000,0.50,1.0,0.333333,0.849581,0.000000,0.036750,0.083333,0.090909,0.618155,0.923077,0.000000,1.051927e-07,2.850000e-08,0.000103,1.000000,0.0,0.000000,0.000000,0.000000,0.979167,0.020833,0.000000,0.015791
201914,2017-08-01,C_ID_7666735b3d,0.75,1.0,0.0,0.093494,0.080560,3,0.094784,0.080560,0.000000,0.129996,0.164699,0.000000,0.833333,0.0,0.833333,0.75,0.0,0.868517,0.931237,0.100000,0.137721,0.750000,0.000000,0.631977,0.923077,0.583333,1.165597e-06,1.124750e-06,0.000308,0.055556,0.0,0.755556,0.000000,0.000000,0.000000,0.866667,0.122222,0.030209
201915,2016-07-01,C_ID_73f5a0efd0,0.50,0.5,1.0,-4.676589,0.253940,3,0.224082,0.169294,0.253940,0.026715,0.031249,0.053448,0.666667,0.6,0.666667,0.50,1.0,0.244017,0.933455,0.000000,0.035047,0.000000,0.090909,0.705677,1.000000,0.333333,7.287418e-07,4.997500e-07,0.000513,1.000000,0.0,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.009955


In [68]:
df_train.isnull().sum()

first_active_month      0
card_id                 0
feature_1               0
feature_2               0
feature_3               0
target                  0
elapsed_time            0
quarter                 0
days_feature1           0
days_feature2           0
days_feature3           0
days_feature1_ratio     1
days_feature2_ratio     1
days_feature3_ratio     1
feature_sum             0
feature_product         0
feature_mean            0
feature_max             0
feature_min             0
feature_var             0
authorized_ratio        0
cat_1_mean              0
installments_mean       0
installments_max        0
installments_min        0
month_lag_mean          0
month_lag_max           0
month_lag_min           0
purchase_amount_mean    0
purchase_amount_max     0
purchase_amount_min     0
cat_2_1_mean            0
cat_2_2_mean            0
cat_2_3_mean            0
cat_2_4_mean            0
cat_2_5_mean            0
cat_3_A_mean            0
cat_3_B_mean            0
cat_3_C_mean

In [69]:
df_test

,first_active_month,card_id,feature_1,feature_2,feature_3,elapsed_time,quarter,days_feature1,days_feature2,days_feature3,days_feature1_ratio,days_feature2_ratio,days_feature3_ratio,feature_sum,feature_product,feature_mean,feature_max,feature_min,feature_var,authorized_ratio,cat_1_mean,installments_mean,installments_max,installments_min,month_lag_mean,month_lag_max,month_lag_min,purchase_amount_mean,purchase_amount_max,purchase_amount_min,cat_2_1_mean,cat_2_2_mean,cat_2_3_mean,cat_2_4_mean,cat_2_5_mean,cat_3_A_mean,cat_3_B_mean,cat_3_C_mean,purchase_freq
0,2017-04-01,C_ID_0ab67a22ab,0.50,1.0,1.0,0.122059,0.333333,0.114688,0.131778,0.133975,0.055589,0.097223,0.101307,0.833333,0.9,0.833333,0.50,1.0,0.333333,0.642470,0.323944,0.188482,1.000000,0.181818,0.642463,1.000000,0.416667,0.000032,0.000007,0.000619,0.676056,0.0,0.000000,0.000000,0.000000,0.0,0.676056,0.323944,0.012366
1,2017-01-01,C_ID_130fd0cbdd,0.25,1.0,0.0,0.162006,0.000000,0.098397,0.171891,0.000000,0.025955,0.074094,0.000000,0.500000,0.0,0.500000,0.50,0.0,0.548584,0.987843,0.045977,0.108218,0.333333,0.181818,0.214677,1.000000,0.000000,0.000024,0.000008,0.001238,0.000000,0.0,0.011494,0.942529,0.000000,0.0,0.965517,0.034483,0.015233
2,2017-08-01,C_ID_b709037bc5,1.00,0.0,1.0,0.067909,0.666667,0.114947,0.022731,0.080560,0.163878,0.051870,0.168478,0.833333,0.5,0.833333,1.00,1.0,1.000000,0.717949,0.133333,0.320928,0.833333,0.000000,0.770122,0.916667,0.583333,0.000231,0.000024,0.004643,0.066667,0.0,0.000000,0.000000,0.800000,0.0,0.133333,0.666667,0.002330
3,2017-12-01,C_ID_d27d835a9f,0.25,0.0,0.0,0.013759,1.000000,0.012025,0.004606,0.000000,0.195575,0.162879,0.000000,0.166667,0.0,0.166667,0.25,0.0,0.244017,1.000000,0.027778,0.172425,1.000000,0.000000,0.854274,1.000000,0.916667,0.000045,0.000006,0.003095,0.944444,0.0,0.000000,0.000000,0.027778,0.0,0.611111,0.333333,0.006093
4,2015-12-01,C_ID_2b5e3df5c2,1.00,0.0,1.0,0.338216,1.000000,0.508663,0.113208,0.347198,0.033777,0.008545,0.039092,0.833333,0.5,0.833333,1.00,1.0,1.000000,0.790285,0.000000,0.107982,0.333333,0.000000,0.461400,1.000000,0.000000,0.000233,0.000122,0.000000,0.034483,0.0,0.051724,0.913793,0.000000,0.0,0.931034,0.060345,0.020430
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123618,2017-10-01,C_ID_7a239d2eda,0.25,0.5,0.0,0.040834,1.000000,0.027799,0.031942,0.000000,0.095839,0.164241,0.000000,0.333333,0.0,0.333333,0.25,0.0,0.333333,0.956533,0.000000,0.019352,0.000000,0.090909,0.762699,0.833333,0.750000,0.000015,0.000002,0.003095,0.041096,0.0,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.012724
123619,2017-09-01,C_ID_75ace375ae,0.50,0.0,1.0,0.054150,0.666667,0.055340,0.018125,0.066988,0.116709,0.063300,0.202614,0.500000,0.3,0.500000,0.50,1.0,0.333333,1.000000,0.000000,0.019352,0.000000,0.090909,0.799230,1.000000,0.666667,0.000012,0.000002,0.001238,0.909091,0.0,0.000000,0.000000,0.090909,1.0,0.000000,0.000000,0.001613
123620,2016-09-01,C_ID_21d56d950c,1.00,0.0,1.0,0.216156,0.666667,0.330877,0.072352,0.226795,0.054645,0.015494,0.059846,0.833333,0.5,0.833333,1.00,1.0,1.000000,0.971414,0.081081,0.150504,0.583333,0.000000,0.609695,0.833333,0.250000,0.000124,0.000018,0.009904,0.027027,0.0,0.000000,0.000000,0.891892,0.0,0.675676,0.297297,0.006272
123621,2017-06-01,C_ID_6c46fc5a9d,0.25,0.0,0.0,0.094984,0.333333,0.059348,0.031793,0.000000,0.045360,0.037824,0.000000,0.166667,0.0,0.166667,0.25,0.0,0.244017,0.704468,0.014706,0.170543,0.416667,0.000000,0.755271,0.916667,0.750000,0.000025,0.000005,0.001857,0.058824,0.0,0.926471,0.000000,0.000000,0.0,0.470588,0.500000,0.011828


In [70]:
df_test.isnull().sum()

first_active_month      1
card_id                 0
feature_1               0
feature_2               0
feature_3               0
elapsed_time            1
quarter                 1
days_feature1           1
days_feature2           1
days_feature3           1
days_feature1_ratio     1
days_feature2_ratio     1
days_feature3_ratio     1
feature_sum             0
feature_product         0
feature_mean            0
feature_max             0
feature_min             0
feature_var             0
authorized_ratio        0
cat_1_mean              0
installments_mean       0
installments_max        0
installments_min        0
month_lag_mean          0
month_lag_max           0
month_lag_min           0
purchase_amount_mean    0
purchase_amount_max     0
purchase_amount_min     0
cat_2_1_mean            0
cat_2_2_mean            0
cat_2_3_mean            0
cat_2_4_mean            0
cat_2_5_mean            0
cat_3_A_mean            0
cat_3_B_mean            0
cat_3_C_mean            0
purchase_fre

In [71]:
path_train_prepro = '../Kaggle-data/processed/train-processed_0624.csv'
path_test_preprp = '../Kaggle-data/processed/test-processed_0624.csv'

df_train.to_csv(path_train_prepro, index=False)
df_test.to_csv(path_test_preprp, index=False)
winsound.Beep(frequency, duration)

In [72]:
df_train

,first_active_month,card_id,feature_1,feature_2,feature_3,target,elapsed_time,quarter,days_feature1,days_feature2,days_feature3,days_feature1_ratio,days_feature2_ratio,days_feature3_ratio,feature_sum,feature_product,feature_mean,feature_max,feature_min,feature_var,authorized_ratio,cat_1_mean,installments_mean,installments_max,installments_min,month_lag_mean,month_lag_max,month_lag_min,purchase_amount_mean,purchase_amount_max,purchase_amount_min,cat_2_1_mean,cat_2_2_mean,cat_2_3_mean,cat_2_4_mean,cat_2_5_mean,cat_3_A_mean,cat_3_B_mean,cat_3_C_mean,purchase_freq
0,2017-06-01,C_ID_92a2005557,1.00,0.5,1.0,-0.820283,0.107268,2,0.157759,0.071512,0.107268,0.121699,0.080192,0.126531,1.000000,1.0,1.000000,1.00,1.0,0.868517,0.952619,0.000000,0.036203,0.083333,0.090909,0.639443,1.000000,0.416667,3.544876e-07,4.997500e-07,0.000256,0.989399,0.0,0.000000,0.000000,0.010601,0.985866,0.014134,0.000000,0.096464
1,2017-01-01,C_ID_3d0044924f,0.75,0.0,0.0,0.392913,0.173380,1,0.203992,0.057793,0.000000,0.057441,0.021668,0.000000,0.500000,0.0,0.500000,0.75,0.0,0.868517,0.968129,0.087079,0.161156,0.833333,0.000000,0.535782,1.000000,0.083333,4.528911e-07,8.942500e-07,0.000154,0.912921,0.0,0.000000,0.000000,0.000000,0.000000,0.792135,0.202247,0.121524
2,2016-08-01,C_ID_d639edf6cd,0.25,0.5,0.0,0.688056,0.240368,3,0.141404,0.160245,0.000000,0.017180,0.033270,0.000000,0.333333,0.0,0.333333,0.25,0.0,0.333333,0.953116,0.000000,0.035047,0.000000,0.090909,0.289328,1.000000,0.000000,2.107272e-07,9.975000e-08,0.000564,0.090909,0.0,0.000000,0.000000,0.909091,1.000000,0.000000,0.000000,0.014418
3,2017-09-01,C_ID_186d6a6901,0.75,1.0,0.0,0.142495,0.066988,3,0.078815,0.066988,0.000000,0.157456,0.198990,0.000000,0.833333,0.0,0.833333,0.75,0.0,0.868517,1.000000,0.154762,0.121690,0.250000,0.000000,0.712551,1.000000,0.666667,3.189285e-07,3.645000e-07,0.000205,0.142857,0.0,0.000000,0.702381,0.000000,0.000000,0.880952,0.083333,0.028150
4,2017-11-01,C_ID_cdbd2c0db2,0.00,1.0,0.0,-0.159749,0.040280,4,0.011848,0.040280,0.000000,0.062232,0.333943,0.000000,0.333333,0.0,0.333333,0.50,0.0,0.548584,0.969484,0.100592,0.140049,1.000000,0.000000,0.839412,1.000000,0.833333,1.072966e-06,1.320750e-06,0.000000,0.071006,0.0,0.041420,0.781065,0.005917,0.000000,0.946746,0.047337,0.057329
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201912,2017-09-01,C_ID_963962de2c,0.50,0.5,1.0,-2.740821,0.066988,3,0.059111,0.044658,0.066988,0.116709,0.131145,0.202614,0.666667,0.6,0.666667,0.50,1.0,0.244017,0.912217,0.000000,0.036787,0.083333,0.090909,0.744939,0.923077,0.666667,1.154149e-07,2.875000e-08,0.000154,0.297872,0.0,0.702128,0.000000,0.000000,0.978723,0.021277,0.000000,0.015448
201913,2015-10-01,C_ID_1314773c0b,0.50,0.0,1.0,0.312917,0.373905,4,0.329942,0.124635,0.373905,0.016369,0.007610,0.036300,0.500000,0.3,0.500000,0.50,1.0,0.333333,0.849581,0.000000,0.036750,0.083333,0.090909,0.618155,0.923077,0.000000,1.051927e-07,2.850000e-08,0.000103,1.000000,0.0,0.000000,0.000000,0.000000,0.979167,0.020833,0.000000,0.015791
201914,2017-08-01,C_ID_7666735b3d,0.75,1.0,0.0,0.093494,0.080560,3,0.094784,0.080560,0.000000,0.129996,0.164699,0.000000,0.833333,0.0,0.833333,0.75,0.0,0.868517,0.931237,0.100000,0.137721,0.750000,0.000000,0.631977,0.923077,0.583333,1.165597e-06,1.124750e-06,0.000308,0.055556,0.0,0.755556,0.000000,0.000000,0.000000,0.866667,0.122222,0.030209
201915,2016-07-01,C_ID_73f5a0efd0,0.50,0.5,1.0,-4.676589,0.253940,3,0.224082,0.169294,0.253940,0.026715,0.031249,0.053448,0.666667,0.6,0.666667,0.50,1.0,0.244017,0.933455,0.000000,0.035047,0.000000,0.090909,0.705677,1.000000,0.333333,7.287418e-07,4.997500e-07,0.000513,1.000000,0.0,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.009955


In [73]:
df_test

,first_active_month,card_id,feature_1,feature_2,feature_3,elapsed_time,quarter,days_feature1,days_feature2,days_feature3,days_feature1_ratio,days_feature2_ratio,days_feature3_ratio,feature_sum,feature_product,feature_mean,feature_max,feature_min,feature_var,authorized_ratio,cat_1_mean,installments_mean,installments_max,installments_min,month_lag_mean,month_lag_max,month_lag_min,purchase_amount_mean,purchase_amount_max,purchase_amount_min,cat_2_1_mean,cat_2_2_mean,cat_2_3_mean,cat_2_4_mean,cat_2_5_mean,cat_3_A_mean,cat_3_B_mean,cat_3_C_mean,purchase_freq
0,2017-04-01,C_ID_0ab67a22ab,0.50,1.0,1.0,0.122059,0.333333,0.114688,0.131778,0.133975,0.055589,0.097223,0.101307,0.833333,0.9,0.833333,0.50,1.0,0.333333,0.642470,0.323944,0.188482,1.000000,0.181818,0.642463,1.000000,0.416667,0.000032,0.000007,0.000619,0.676056,0.0,0.000000,0.000000,0.000000,0.0,0.676056,0.323944,0.012366
1,2017-01-01,C_ID_130fd0cbdd,0.25,1.0,0.0,0.162006,0.000000,0.098397,0.171891,0.000000,0.025955,0.074094,0.000000,0.500000,0.0,0.500000,0.50,0.0,0.548584,0.987843,0.045977,0.108218,0.333333,0.181818,0.214677,1.000000,0.000000,0.000024,0.000008,0.001238,0.000000,0.0,0.011494,0.942529,0.000000,0.0,0.965517,0.034483,0.015233
2,2017-08-01,C_ID_b709037bc5,1.00,0.0,1.0,0.067909,0.666667,0.114947,0.022731,0.080560,0.163878,0.051870,0.168478,0.833333,0.5,0.833333,1.00,1.0,1.000000,0.717949,0.133333,0.320928,0.833333,0.000000,0.770122,0.916667,0.583333,0.000231,0.000024,0.004643,0.066667,0.0,0.000000,0.000000,0.800000,0.0,0.133333,0.666667,0.002330
3,2017-12-01,C_ID_d27d835a9f,0.25,0.0,0.0,0.013759,1.000000,0.012025,0.004606,0.000000,0.195575,0.162879,0.000000,0.166667,0.0,0.166667,0.25,0.0,0.244017,1.000000,0.027778,0.172425,1.000000,0.000000,0.854274,1.000000,0.916667,0.000045,0.000006,0.003095,0.944444,0.0,0.000000,0.000000,0.027778,0.0,0.611111,0.333333,0.006093
4,2015-12-01,C_ID_2b5e3df5c2,1.00,0.0,1.0,0.338216,1.000000,0.508663,0.113208,0.347198,0.033777,0.008545,0.039092,0.833333,0.5,0.833333,1.00,1.0,1.000000,0.790285,0.000000,0.107982,0.333333,0.000000,0.461400,1.000000,0.000000,0.000233,0.000122,0.000000,0.034483,0.0,0.051724,0.913793,0.000000,0.0,0.931034,0.060345,0.020430
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123618,2017-10-01,C_ID_7a239d2eda,0.25,0.5,0.0,0.040834,1.000000,0.027799,0.031942,0.000000,0.095839,0.164241,0.000000,0.333333,0.0,0.333333,0.25,0.0,0.333333,0.956533,0.000000,0.019352,0.000000,0.090909,0.762699,0.833333,0.750000,0.000015,0.000002,0.003095,0.041096,0.0,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.012724
123619,2017-09-01,C_ID_75ace375ae,0.50,0.0,1.0,0.054150,0.666667,0.055340,0.018125,0.066988,0.116709,0.063300,0.202614,0.500000,0.3,0.500000,0.50,1.0,0.333333,1.000000,0.000000,0.019352,0.000000,0.090909,0.799230,1.000000,0.666667,0.000012,0.000002,0.001238,0.909091,0.0,0.000000,0.000000,0.090909,1.0,0.000000,0.000000,0.001613
123620,2016-09-01,C_ID_21d56d950c,1.00,0.0,1.0,0.216156,0.666667,0.330877,0.072352,0.226795,0.054645,0.015494,0.059846,0.833333,0.5,0.833333,1.00,1.0,1.000000,0.971414,0.081081,0.150504,0.583333,0.000000,0.609695,0.833333,0.250000,0.000124,0.000018,0.009904,0.027027,0.0,0.000000,0.000000,0.891892,0.0,0.675676,0.297297,0.006272
123621,2017-06-01,C_ID_6c46fc5a9d,0.25,0.0,0.0,0.094984,0.333333,0.059348,0.031793,0.000000,0.045360,0.037824,0.000000,0.166667,0.0,0.166667,0.25,0.0,0.244017,0.704468,0.014706,0.170543,0.416667,0.000000,0.755271,0.916667,0.750000,0.000025,0.000005,0.001857,0.058824,0.0,0.926471,0.000000,0.000000,0.0,0.470588,0.500000,0.011828
